# Summary

# Imports

In [ ]:
import os
import shutil
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
pd.set_option("max_columns", 100)

# Parameters

In [ ]:
NOTEBOOK_PATH = Path('train_network')
NOTEBOOK_PATH

In [ ]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH

In [ ]:
DEBUG = "CI" not in os.environ
if DEBUG:
    assert "SLURM_JOB_ID" not in os.environ
DEBUG

# Train network

In [ ]:
DATAPKG_OUTPUT_PATH = Path(os.environ['DATAPKG_OUTPUT_DIR'])
DATAPKG_OUTPUT_PATH

In [ ]:
def parse_slurm_runtime(sbatch_timelimit: str) -> float:
    runtime = 0
    multipliers = [3600, 60, 1]
    while sbatch_timelimit:
        sbatch_timelimit, _, value = sbatch_timelimit.rpartition(':')
        multipier = multipliers.pop()
        if value:
            runtime += int(value) * multipier
    return runtime

            
parse_slurm_runtime("1:20:11")

In [ ]:
if DEBUG:
    runtime = 60  # 1m
else:
    runtime = parse_slurm_runtime(os.environ['SBATCH_TIMELIMIT']) - 1800  # Total time - 30m
runtime

In [ ]:
from pagnn.training.dcn import Args, main

In [ ]:
args = Args(
    root_path=OUTPUT_PATH,
    training_data_path=(
        DATAPKG_OUTPUT_PATH
        .joinpath("adjacency-net-v2", "master", "training_dataset", "adjacency_matrix.parquet")
    ),
#     training_data_cache=(
#         DATAPKG_OUTPUT_PATH
#         .joinpath("adjacency-net-v2", "master", "training_dataset", f"array_id_{Args().array_id}")
#     ),
    network_name="Custom",
    gpu=-1,
    verbosity=1,
)
args

In [ ]:
assert isinstance(args.runtime, float)
args.runtime = runtime

In [ ]:
%run {NOTEBOOK_PATH}/model.py

In [ ]:
shutil.copy2(NOTEBOOK_PATH.joinpath("model.py"), OUTPUT_PATH.joinpath("model.py"))

In [ ]:
main(args)